<a href="https://colab.research.google.com/github/yinghaow525/MSBA-Projects/blob/main/Capstone%20Project/Capstone_A6_Coding_Deliverable_01_Basic_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capsone Project with Spinnaker Analytics
Team A6: Yihan Jia, Yinghao Wang, Simeng Li, Xiangshan Mu, Kaiyu Wang
> **Business Objectives**:
- The objective of the exercise is to find a tradable signal in the dataset. 

> **Project Flow**:
- Standard time series models can be used or non-parametric models like neural networks can be deployed. Suggested course of actions is as follows:
  1. Understand the dataset and what it represents
  2. Define the problem: what does 'a tradable signal' mean based on the data available (i.e. develop your modeling equation, if any)
  3. Start with simple analysis to identify patterns and outliers
  4. Develop models with increasing complexity - i.e. don't start with neural nets etc
  5. Define your success criteria - i.e. when do you know you have a tradable signal - positive (buy)/negative (sell), both are signals


# Library Imports

In [1]:
# imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

from google.colab import drive

from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.api import ExponentialSmoothing

from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

from statsmodels.tsa.arima_model import ARIMA



from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, neural_network
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support

#reset recursionlimit
import sys
sys.setrecursionlimit(10000)

import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable

# supress filter warnings
import warnings
warnings.filterwarnings("ignore")

#plt style
plt.style.use("seaborn")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Source

The dataset contains sectoral data for 3 separate types of investments made in the US  (no international funds are included) and represents 60-70% of activity in the overall asset classes for that week (not all investors report data at the same time). The weekly data spans 10 years from 2006 through end-Jan 2017. Not all sectors have data available for all the dates since new investment vehicles are introduced at various points in time. 
1. **Institutional Mutual Fund Holdings** (investments made/redeemed by institutional investors like Fidelity, Vanguard on behalf of insitutions like CALPERS etc)
2. **Retail Mutual Fund Holdings** (investments made by individuals in their portfolios) 
3. **Exchange Traded Funds** - insitutional investors


## Data fields & descriptions:
- `ReportDate`: Weekly data aggregated and released every Wednesday	
- `AssetClass`: Industry/Sector/Asset Class	
- `Flow`: Amount of positive (inflow) or negative (outflow) in Millions of USD 	
- `FlowPct`: Flows as percent of assets at beginning of the week 	
- `AssetsEnd`: Assets at end of the week in Millions of USD	
- `PortfolioChangePct`: Percent change in overall portfolio during the week

## Data Imports

In [2]:
# read datasets
# try:
#   # mount my google drive
#   drive.mount('/content/drive')
#   # change directory to the path that contains dataset uploaded
#   %cd /content/drive/My Drive/Capstone Project

#   # read through google drive
#   IMF = pd.read_csv('US Sector Inst ETF.csv')
#   ETF = pd.read_csv('US Sector Inst ETF.csv')
#   RMF  = pd.read_csv('US Sector Retail MF.csv')

# except: 
  # or read through github
ETF = pd.read_csv('https://raw.githubusercontent.com/yinghaow525/MSBA-Projects/main/Capstone%20Project/Datasets/US%20Sector%20Inst%20ETF.csv')
IMF = pd.read_csv('https://raw.githubusercontent.com/yinghaow525/MSBA-Projects/main/Capstone%20Project/Datasets/US%20Sector%20Institutional%20MF.csv')
RMF = pd.read_csv('https://raw.githubusercontent.com/yinghaow525/MSBA-Projects/main/Capstone%20Project/Datasets/US%20Sector%20Retail%20MF.csv')

In [3]:
print(IMF.head(3))
print('-'*50)
print(ETF.head(3))
print('-'*50)
print(RMF.head(3))

             ReportDate                                         AssetClass  \
0  2/1/2017 12:00:00 AM  Commodities/Materials-North America-USA-North ...   
1  2/1/2017 12:00:00 AM  Consumer Goods-North America-USA-North America...   
2  2/1/2017 12:00:00 AM      Energy-North America-USA-North America-Equity   

         Flow  FlowPct     AssetsEnd  PortfolioChangePct  
0  380.126769   4.4676   8787.631717             -1.1853  
1  332.526792   1.1479  28973.613065             -1.1231  
2  217.857886   0.3460  62136.008339             -1.6441  
--------------------------------------------------
             ReportDate                                         AssetClass  \
0  2/1/2017 12:00:00 AM  Commodities/Materials-North America-USA-North ...   
1  2/1/2017 12:00:00 AM  Consumer Goods-North America-USA-North America...   
2  2/1/2017 12:00:00 AM      Energy-North America-USA-North America-Equity   

         Flow  FlowPct     AssetsEnd  PortfolioChangePct  
0  378.578706   4.5064   867

# Concating & Preprocessing

In [4]:
# Create type colunm
ETF['Type'] = 'ETF'
IMF['Type'] = 'IMF'
RMF['Type'] = 'RMF'

# Concat 3 dadaset
data = pd.concat([ETF, IMF, RMF], axis=0)

In [5]:
# check NAs
print(data.isna().sum())

ReportDate            0
AssetClass            0
Flow                  0
FlowPct               0
AssetsEnd             0
PortfolioChangePct    0
Type                  0
dtype: int64


In [6]:
# check duplicates
data.duplicated().sum()

0

In [7]:
# convert report date to pandas datetime object
data['ReportDate'] = pd.to_datetime(data['ReportDate'])

In [8]:
# since all the Asset and Sector values are the same (Equity, and within US)
# we only keep Industry sector as features
data['AssetClass'] = data['AssetClass'].str.split('-')
data['Industry'] = data.AssetClass.apply(lambda x: x[0])

# drop orignal AssetClass feature
data.drop(columns='AssetClass', inplace=True)

# types and number of Industry values
data.Industry.value_counts()

Mid Cap Growth           1737
Technology               1737
Energy                   1737
Financials               1737
Health Care/Biotech      1737
Utilities                1737
Large Cap Blend          1737
Large Cap Growth         1737
Large Cap Value          1737
Mid Cap Blend            1737
Telecom                  1737
Mid Cap Value            1737
Real Estate              1737
Small Cap Blend          1737
Small Cap Growth         1737
Small Cap Value          1737
Commodities/Materials    1700
Consumer Goods           1700
Industrials              1435
Infrastructure             61
Name: Industry, dtype: int64

In [9]:
# extract year & month 
data['year'], data['month'] = data['ReportDate'].dt.year, data['ReportDate'].dt.month
data.head()

,ReportDate,Flow,FlowPct,AssetsEnd,PortfolioChangePct,Type,Industry,year,month
0,2017-02-01,378.578706,4.5064,8679.056347,-1.1938,ETF,Commodities/Materials,2017,2
1,2017-02-01,332.526792,1.1479,28973.613065,-1.1231,ETF,Consumer Goods,2017,2
2,2017-02-01,186.031374,0.3782,48446.700077,-1.8855,ETF,Energy,2017,2
3,2017-02-01,968.885223,1.5139,64378.816729,-0.9202,ETF,Financials,2017,2
4,2017-02-01,410.686628,1.1532,36846.852743,2.3119,ETF,Health Care/Biotech,2017,2


In [10]:
# max & min value for Flow
print('Max value of Flow is {};'.format(data.Flow.max()), '\n'
      'Min value of Flow is {};'.format(data.Flow.min()),'\n'
      'Avg value of Flow is {}.'.format(data.Flow.mean()))

Max value of Flow is 23263.40489835; 
Min value of Flow is -13967.24062477; 
Avg value of Flow is 20.653445520794538.
